In [1]:
import pandas as pd
import pickle
import os

In [1]:
#Prof example install
#%pip install lda

  Created wheel for lda: filename=lda-2.0.0-cp38-cp38-win_amd64.whl size=346338 sha256=9f1a44fcee7b2548716e469505228f906a751b0e584cd06bdcaf804a19283a87
  Stored in directory: c:\users\louis\appdata\local\pip\cache\wheels\bf\64\c8\595c54dbccb82a2c7e46b0759030acc8770781a6ec547e0c53
Successfully built lda
Note: you may need to restart the kernel to use updated packages.


In [133]:
#Prof example import 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import lda
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [217]:
df_lyrics = pd.read_pickle("df_rap_provisional.pickle")

In [218]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_wordlen,genre_rap,genre_hip_hop,track_release_year,track_release_decade
0,5DJNyvuMmZfsfvdTpMMmUq,08IW6sUfN9fRYG8Xlgm4oC,1ShZZUjkbXCjhwrb18BA8I,252546,True,https://api.spotify.com/v1/tracks/5DJNyvuMmZfs...,Sour Patch Kids,70,track,0.523,...,431943,71,"When I was growing up, I had a lot of dreams, ...",growing lot dreams momma told son could anythi...,"[growing, lot, dreams, momma, told, son, could...",251,1,0,2014,10
1,1uZXf9hQR1zkh1TPIGbuFP,6O24vHd68xPWQZGU9ppv7G,4hxDvVq5t8ebPYPdBl1F9f,274384,True,https://api.spotify.com/v1/tracks/1uZXf9hQR1zk...,Sangre Por Sangre (Blood For Blood),41,track,0.509,...,571234,55,"Sangre por sangre, blood for blood\nSangre por...",sangre por sangre blood blood sangre por sangr...,"[sangre, por, sangre, blood, blood, sangre, po...",206,1,0,2014,10
2,4MJfCOcc2PpWOrUqOpvy1S,5UOImThFBmntV6lpCllNFU,1ShZZUjkbXCjhwrb18BA8I,228610,True,https://api.spotify.com/v1/tracks/4MJfCOcc2PpW...,Sunflower Seeds,63,track,0.723,...,431943,71,40 ounces and a pack of ranch sunflower seeds\...,ounces pack ranch sunflower seeds answers ques...,"[ounces, pack, ranch, sunflower, seeds, answer...",212,1,0,2016,10
3,0fCwrjKfWxF3xQhZrYUw4T,6n9DKpOxwifT5hOXtgLZSL,4MCBfE4596Uoi2O4DtmEMz,154767,True,https://api.spotify.com/v1/tracks/0fCwrjKfWxF3...,Blood On My Jeans,75,track,0.868,...,21026508,94,"Ah\nDamn, I tried to stop, oh\nBaby (808 Mafia...",damn tried stop baby mafia baby baby baby baby...,"[damn, tried, stop, baby, mafia, baby, baby, b...",242,2,0,2020,20
4,0P9lyICk63dqZSqXV2HBP0,24i8TDxRtaHGSDiWjVyseC,4qXC0i02bSFstECuXP2ZpL,170702,True,https://api.spotify.com/v1/tracks/0P9lyICk63dq...,About You,67,track,0.644,...,227241,76,Ayy\n\nI need you to be about we (We)\nIf you ...,ayy need put limit love count girl know selfis...,"[ayy, need, put, limit, love, count, girl, kno...",212,1,0,2021,20


In [219]:

wordnet_lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [220]:
temp = df_lyrics["lyrics_clean"]

In [221]:
words = set(nltk.corpus.words.words())

def remove_foreign(t):
    return " ".join(w for w in nltk.wordpunct_tokenize(t) if w.lower() in words or not w.isalpha())

In [222]:
df_lyrics["lyrics_clean"] = temp.apply(remove_foreign)
df_lyrics["lyrics_clean"]

0       growing lot told son could anything long sprea...
1       blood blood blood blood blood blood blood bloo...
2       pack ranch sunflower later got nowhere crew up...
3       damn tried stop baby baby baby baby baby babe ...
4       need put limit love count girl know selfish wa...
                              ...                        
2462    hey hey mike made hey hey hey done seen floor ...
2463    think time think may need mine hate say know l...
2464    yeah yeah yeah yeah yeah say love know forget ...
2465    must thousand million hear moving street read ...
2466    feel better hold one get bumpy ten hell bent h...
Name: lyrics_clean, Length: 2467, dtype: object

In [223]:
def tokenize_text(raw_text):
    text = raw_text.lower()
    tokens = word_tokenizer.tokenize(text)
    return tokens

In [301]:
def tokenize_text_lemma(raw_text):
    lowercase = raw_text.lower()  
    tokens = word_tokenizer.tokenize(lowercase)
    lemmed_text = [wordnet_lemmatizer.lemmatize(i) for i in tokens]

    return lemmed_text

In [307]:
def tokenize_text_stem(raw_text):
    lowercase = raw_text.lower()
    tokens = word_tokenizer.tokenize(lowercase)
    stemmed_text = [stemmer.stem(i) for i in tokens ]

    return stemmed_text

In [308]:
example = "Yo what is up reliable reliability reliably car cars good better best"

In [309]:
tokenize_text_stem(example)

['yo',
 'what',
 'is',
 'up',
 'reliabl',
 'reliabl',
 'reliabl',
 'car',
 'car',
 'good',
 'better',
 'best']

In [305]:
tokenize_text_lemma(example)

['yo',
 'what',
 'is',
 'up',
 'reliable',
 'reliability',
 'reliably',
 'car',
 'car',
 'good',
 'better',
 'best']

In [310]:
vec_words = CountVectorizer(tokenizer = tokenize_text_lemma, stop_words = stopwords_nltk, decode_error="ignore")
total_words = vec_words.fit_transform(df_lyrics["lyrics_clean"])

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn(


In [312]:
print(total_words.shape)


(2467, 10842)


In [313]:
#Number of topics to be estimated 
ntopics = 10

In [314]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_words)

INFO:lda:n_documents: 2467
INFO:lda:vocab_size: 10842
INFO:lda:n_words: 405414
INFO:lda:n_topics: 10
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -3985906
INFO:lda:<10> log likelihood: -3257629
INFO:lda:<20> log likelihood: -3153222
INFO:lda:<30> log likelihood: -3115668
INFO:lda:<40> log likelihood: -3096341
INFO:lda:<50> log likelihood: -3082102
INFO:lda:<60> log likelihood: -3071116
INFO:lda:<70> log likelihood: -3064407
INFO:lda:<80> log likelihood: -3057926
INFO:lda:<90> log likelihood: -3055260
INFO:lda:<100> log likelihood: -3050612
INFO:lda:<110> log likelihood: -3047114
INFO:lda:<120> log likelihood: -3042921
INFO:lda:<130> log likelihood: -3038387
INFO:lda:<140> log likelihood: -3037414
INFO:lda:<150> log likelihood: -3035824
INFO:lda:<160> log likelihood: -3034002
INFO:lda:<170> log likelihood: -3033016
INFO:lda:<180> log likelihood: -3030670
INFO:lda:<190> log likelihood: -3028765
INFO:lda:<200> log likelihood: -3027650
INFO:lda:<210> log likelihood: -3027388
INFO:lda:

In [315]:
topic_word = model.topic_word_


In [316]:
doc_topic = model.doc_topic_


In [317]:
doc_topic = pd.DataFrame(doc_topic)


In [318]:
#Get a copy of df_lyrics for join 
df_lyrics_copy = df_lyrics.copy()

In [319]:
df_lyrics_copy = df_lyrics_copy.join(doc_topic)


In [320]:
df_lyrics_copy.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,0,1,2,3,4,5,6,7,8,9
0,5DJNyvuMmZfsfvdTpMMmUq,08IW6sUfN9fRYG8Xlgm4oC,1ShZZUjkbXCjhwrb18BA8I,252546,True,https://api.spotify.com/v1/tracks/5DJNyvuMmZfs...,Sour Patch Kids,70,track,0.523,...,0.000578,0.012139,0.041040,0.000578,0.069942,0.000578,0.445665,0.110405,0.318497,0.000578
1,1uZXf9hQR1zkh1TPIGbuFP,6O24vHd68xPWQZGU9ppv7G,4hxDvVq5t8ebPYPdBl1F9f,274384,True,https://api.spotify.com/v1/tracks/1uZXf9hQR1zk...,Sangre Por Sangre (Blood For Blood),41,track,0.509,...,0.000581,0.087791,0.000581,0.000581,0.198256,0.000581,0.093605,0.616860,0.000581,0.000581
2,4MJfCOcc2PpWOrUqOpvy1S,5UOImThFBmntV6lpCllNFU,1ShZZUjkbXCjhwrb18BA8I,228610,True,https://api.spotify.com/v1/tracks/4MJfCOcc2PpW...,Sunflower Seeds,63,track,0.723,...,0.000613,0.092638,0.080368,0.196933,0.019018,0.031288,0.147853,0.190798,0.184663,0.055828
3,0fCwrjKfWxF3xQhZrYUw4T,6n9DKpOxwifT5hOXtgLZSL,4MCBfE4596Uoi2O4DtmEMz,154767,True,https://api.spotify.com/v1/tracks/0fCwrjKfWxF3...,Blood On My Jeans,75,track,0.868,...,0.256784,0.467839,0.000503,0.000503,0.000503,0.000503,0.020603,0.000503,0.176382,0.075879
4,0P9lyICk63dqZSqXV2HBP0,24i8TDxRtaHGSDiWjVyseC,4qXC0i02bSFstECuXP2ZpL,170702,True,https://api.spotify.com/v1/tracks/0P9lyICk63dq...,About You,67,track,0.644,...,0.072778,0.089444,0.000556,0.000556,0.000556,0.000556,0.061667,0.000556,0.350556,0.422778


In [321]:
artists = pd.DataFrame()

In [322]:
for i in range(int(ntopics)):
    topic="topic_"+str(i)
    artists[topic]=df_lyrics_copy.groupby(["artist_name"])[i].mean()

In [323]:
artists = artists.reset_index()

In [324]:
topics = pd.DataFrame(topic_word)


In [325]:
topics.columns = vec_words.get_feature_names_out()

In [326]:
topics_transpose = topics.transpose()


In [327]:
topics_transpose

,0,1,2,3,4,5,6,7,8,9
aback,2.991288e-07,1.115462e-05,6.547751e-07,6.047624e-07,1.757796e-07,4.842517e-07,1.990112e-07,3.961428e-07,1.435041e-07,3.571630e-07
abacus,2.991288e-07,1.104418e-07,6.547751e-07,6.047624e-07,1.757796e-07,4.842517e-07,1.990112e-07,3.961428e-07,2.884432e-05,3.571630e-07
abandon,2.991288e-07,1.104418e-07,2.625648e-04,6.047624e-07,1.757796e-07,4.842517e-07,1.990112e-07,3.961428e-07,1.435041e-07,3.571630e-07
abandoned,2.991288e-07,1.104418e-07,6.547751e-07,6.047624e-07,1.757796e-07,4.842517e-07,2.788147e-04,1.192390e-04,1.435041e-07,3.571630e-07
abhor,2.991288e-07,1.104418e-07,6.547751e-07,6.047624e-07,1.757796e-07,4.842517e-07,1.990112e-07,1.192390e-04,1.435041e-07,3.571630e-07
...,...,...,...,...,...,...,...,...,...,...
zone,5.686438e-04,1.104418e-07,1.834025e-03,6.047624e-07,1.757796e-07,4.842517e-07,5.972327e-04,3.961428e-07,1.435041e-07,3.571630e-07
zoned,2.991288e-07,1.104418e-07,6.547751e-07,6.047624e-07,1.757796e-07,9.733458e-05,1.990112e-07,3.961428e-07,1.435041e-07,3.571630e-07
zoning,2.991288e-07,1.115462e-05,6.547751e-07,6.047624e-07,1.757796e-07,4.842517e-07,1.990112e-07,3.961428e-07,1.435041e-07,3.571630e-07
zoo,2.991288e-07,4.428717e-05,3.280423e-04,6.047624e-07,1.757796e-07,4.842517e-07,1.990112e-07,3.961428e-07,1.435041e-07,3.571630e-07


In [329]:
#Excel exports if we want to

#topics_transpose.to_excel("topic_word_dist.xlsx")
#atists.to_excel("artists_topic_dist.xlsx",index=False)

In [330]:
#Most important words for topic 0
topics_transpose.iloc[:,0].sort_values(ascending=False).head(40)

yeah     0.344268
huh      0.032755
like     0.022016
bitch    0.020072
got      0.014598
woo      0.014329
money    0.009213
get      0.008765
hold     0.008705
want     0.008585
right    0.008226
pull     0.008107
let      0.007778
baby     0.007239
head     0.006910
damn     0.006850
top      0.006521
bad      0.006461
tell     0.006192
know     0.006132
back     0.005265
pop      0.005026
god      0.004966
came     0.004966
made     0.004876
new      0.004786
rock     0.004487
yuh      0.004457
keep     0.004248
man      0.004098
took     0.004068
yah      0.003769
real     0.003769
party    0.003709
hit      0.003709
hey      0.003620
price    0.003590
hell     0.003530
wait     0.003530
need     0.003440
Name: 0, dtype: float64

In [331]:
#Most important words for topic 1
topics_transpose.iloc[:,1].sort_values(ascending=False).head(30)

like     0.042034
bitch    0.036048
got      0.032757
get      0.018057
money    0.014170
gon      0.011884
know     0.011674
put      0.008869
big      0.007532
hit      0.006958
back     0.006682
gang     0.006616
cause    0.006339
bout     0.006273
keep     0.006262
make     0.006218
run      0.006163
boy      0.006130
call     0.005831
still    0.005169
see      0.004815
new      0.004815
pussy    0.004782
young    0.004771
two      0.004760
rich     0.004694
need     0.004617
dick     0.004572
give     0.004561
come     0.004484
Name: 1, dtype: float64

In [332]:
#Most important words for topic 2
topics_transpose.iloc[:,2].sort_values(ascending=False).head(30)

hey       0.069931
come      0.035686
say       0.024227
turn      0.016632
feat      0.016043
alive     0.015584
push      0.014799
sweet     0.013620
take      0.012310
fire      0.012310
music     0.011066
blue      0.009953
ocean     0.009888
look      0.009822
hello     0.009168
dance     0.008971
side      0.008644
keep      0.008644
life      0.008251
cha       0.008054
til       0.007923
back      0.007007
shake     0.006941
frank     0.006876
hot       0.006679
ignite    0.006614
sex       0.006548
see       0.006483
please    0.005959
lady      0.005763
Name: 2, dtype: float64

In [333]:
#Most important words for topic 3
topics_transpose.iloc[:,3].sort_values(ascending=False).head(20)

get        0.071846
little     0.023526
got        0.016873
round      0.016329
real       0.016208
take       0.014031
one        0.012277
bit        0.011854
four       0.011552
five       0.009979
white      0.009737
come       0.009677
six        0.009072
three      0.008709
man        0.008467
drink      0.008407
give       0.007681
pop        0.007560
nothing    0.007560
two        0.007379
Name: 3, dtype: float64

In [334]:
#Most important words for topic 4
topics_transpose.iloc[:,4].sort_values(ascending=False).head(20)

feel      0.024715
away      0.021463
life      0.014537
know      0.014344
love      0.013148
heart     0.012322
let       0.012024
like      0.011936
one       0.011619
never     0.011320
see       0.011320
take      0.011022
way       0.010740
need      0.010125
inside    0.010055
give      0.009615
day       0.009510
time      0.009141
find      0.008684
pain      0.008033
Name: 4, dtype: float64

In [335]:
#Most important words for topic 5
topics_transpose.iloc[:,5].sort_values(ascending=False).head(20)

let          0.054818
night        0.033704
whoa         0.022615
get          0.021986
know         0.018789
tonight      0.017821
right        0.015400
like         0.014141
everybody    0.014092
work         0.013560
play         0.013027
check        0.012833
high         0.012639
jump         0.011526
party        0.010460
around       0.010315
good         0.010073
alright      0.009637
throw        0.009540
keep         0.008959
Name: 5, dtype: float64

In [336]:
#Most important words for topic 6
topics_transpose.iloc[:,6].sort_values(ascending=False).head(20)

man       0.021394
got       0.020757
god       0.018051
like      0.016120
back      0.011861
know      0.010846
still     0.010468
life      0.010389
get       0.009831
see       0.009692
time      0.008936
day       0.008538
one       0.008080
new       0.007841
make      0.007224
tell      0.006587
look      0.006289
people    0.005971
yeah      0.005951
lot       0.005891
Name: 6, dtype: float64

In [337]:
#Most important words for topic 7
topics_transpose.iloc[:,7].sort_values(ascending=False).head(20)

head       0.018104
keep       0.017233
want       0.016520
mind       0.016480
world      0.015727
dead       0.014618
die        0.012083
know       0.011251
hate       0.010538
running    0.010221
break      0.010062
stay       0.009666
wait       0.009547
cause      0.009151
see        0.009032
blood      0.008082
done       0.007567
kill       0.006774
murder     0.005982
life       0.005863
Name: 7, dtype: float64

In [338]:
artists.head(20)

,artist_name,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,$NOT,0.158674,0.361561,0.012567,0.018927,0.181608,0.021825,0.036360,0.058357,0.117131,0.032990
1,$ilkMoney,0.202083,0.493750,0.014583,0.000694,0.000694,0.000694,0.104861,0.000694,0.181250,0.000694
2,$uicideboy$,0.066304,0.055435,0.001087,0.001087,0.251087,0.077174,0.077174,0.316304,0.153261,0.001087
3,070 Shake,0.131079,0.027012,0.055031,0.022349,0.282459,0.055425,0.046429,0.006220,0.191312,0.182684
4,10 Years,0.001619,0.017004,0.033459,0.030192,0.480001,0.026982,0.057799,0.282080,0.041465,0.029397
5,12 Stones,0.000909,0.100909,0.000909,0.000909,0.491818,0.000909,0.000909,0.400909,0.000909,0.000909
6,2 Chainz,0.233701,0.402541,0.000437,0.052029,0.036195,0.066802,0.148163,0.028586,0.027760,0.003787
7,21 Savage,0.121006,0.429162,0.012349,0.038212,0.050706,0.010500,0.230216,0.024548,0.060318,0.022984
8,22Gz,0.000474,0.834597,0.000474,0.000474,0.000474,0.000474,0.066825,0.095261,0.000474,0.000474
9,24kGoldn,0.002922,0.224520,0.007489,0.009772,0.096699,0.007489,0.096060,0.056638,0.497774,0.000639
